In [24]:
import pandas as pd 
import numpy as np
import math as m
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import random
import pylab as pl

from sklearn.svm import SVC
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold

In [25]:
data = pd.read_csv('690a-vazao-vila-aurea-20-04-2016.csv')
#data = pd.read_csv('690g-pressao_entrada-vila-aurea-20-04-2016.csv')

data_matrix = data.as_matrix()

data_hora = data_matrix[340519:,1:2]

data_matrix = data_matrix[340519:,2:]   

minutos = data_matrix.shape[0]

data_matrix = np.array(data_matrix).ravel()

#data_matrix = np.nan_to_num(data_matrix) 

In [26]:
# Eliminando os buracos
for i in range(minutos):
    if (pd.isnull(data_matrix[i])):
        j = i
        while(pd.isnull(data_matrix[i])):
            i += 1      
            
        data_matrix[j:i] = (data_matrix[j - 1] + data_matrix[i])/2   

In [27]:
# smoothing an array that repreimport pandas as pd 
import numpy as np
import math as m
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches 
def hanning(x, window_len):
    #x_=np.r_[2*x[0]-x[window_len:1:-1], x, 2*x[-1]-x[-1:-window_len:-1]]
    # Extend input data to avoid border issues.
    x_ = np.r_[[x[0]]*(window_len-1), x, [x[-1]]*(window_len-1)]
    # Hanning window (filter).
    f = np.hanning(window_len)
    # Apply convolution.
    y = np.convolve(x_ , f/f.sum(), mode='same')
    # Return data to original size.
    return y[window_len-1:-window_len+1]

In [28]:
data_matrix_ruido = hanning(data_matrix, 10) # Dados da vazão suavizados

data_matrix_suavizado = np.copy(data_matrix_ruido)

In [29]:
# Gerar 200 pontos negativos (sem ruido), 100 para treino e 100 para teste

n_pontos_negativos = 200
minutos_sem_ruido = 60

pontos_negativos = np.empty((n_pontos_negativos, minutos_sem_ruido))

for i in range(n_pontos_negativos):
    inicio = np.random.randint(0, minutos - minutos_sem_ruido)
    pontos_negativos[i,] = data_matrix_ruido[inicio: inicio + minutos_sem_ruido]

pontos_negativos = pontos_negativos.reshape(2,-1)

print pontos_negativos.shape

(2, 6000)


In [30]:
# Gerar 200 pontos positivos (com ruido), 100 para treino e 100 para teste 

n_pontos_positivos = 200  
minutos_ruido = 60
pontos_positivos = np.empty((n_pontos_positivos, minutos_ruido))

d = np.std(data_matrix)                                        # Desvio padrão de todo o data_matrix                          
print "d = %f"%(d)
 
inicio_ruido = np.random.randint(0, minutos, n_pontos_positivos) # Gera n índices para iniciar o ruido

duracao = np.random.normal(120, 40, n_pontos_positivos).round().astype(np.int)  # Gera n durações de ruidos

std = 0.05 * d
intensidade = np.abs(np.random.normal(d, std, n_pontos_positivos))                      # Gera n intensidades de ruidos

for i in range(n_pontos_positivos):
    data_matrix_ruido[inicio_ruido[i]: inicio_ruido[i] + duracao[i] + 1] += intensidade[i] # Acrescenta a intensidade 
                                                                                           # gerada para cada ruido 

for i in range(n_pontos_positivos):
    pontos_positivos[i,] = data_matrix[inicio_ruido[i] - 30: inicio_ruido[i] + 30]  # Pontos positivos

pontos_positivos = pontos_positivos.reshape(2,-1)

print pontos_positivos.shape

d = 10.825399
(2, 6000)


In [31]:
# Recebe uma matriz de dados e retorna esta mesma matriz com ruidos
def data_matrix_ruido (data_matrix):
    
    n_pontos_positivos = 200  
    minutos_ruido = 60
    pontos_positivos = np.empty((n_pontos_positivos, minutos_ruido))

    d = np.std(data_matrix)                                        # Desvio padrão de todo o data_matrix                          
    #print "d = %f"%(d)

    inicio_ruido = np.random.randint(0, minutos, n_pontos_positivos) # Gera n índices para iniciar o ruido

    duracao = np.random.normal(120, 40, n_pontos_positivos).round().astype(np.int)  # Gera n durações de ruidos

    std = 0.05 * d
    intensidade = np.abs(np.random.normal(d, std, n_pontos_positivos))                      # Gera n intensidades de ruidos

    for i in range(n_pontos_positivos):
        data_matrix[inicio_ruido[i]: inicio_ruido[i] + duracao[i] + 1] += intensidade[i] # Acrescenta a intensidade 
                                                                                               # gerada para cada ruido 
    return data_matrix

In [32]:
data_ruido = data_matrix_ruido(data_matrix_suavizado)

print data_ruido

[43.0 43.293128413857275 43.293128413857275 ..., 36.00000000000001
 36.00000000000001 36.00000000000001]


In [33]:
def plot_grafico(data_matrix_suavizado, data_matrix_ruido, inicio_ruido, duracao): 
    
    n = inicio_ruido.shape[0]
    
    for i in range(10):
        x_ruido = range(duracao[i], 2 * duracao[i])
        y_ruido = data_matrix_ruido[inicio_ruido[i]: inicio_ruido[i] + duracao[i]]
        
        x_hann = range(duracao[i], 2 * duracao[i] + 1)
        y_hann = data_matrix_suavizado[inicio_ruido[i]: inicio_ruido[i] + duracao[i] + 1]
            
        x_ruido_completo = range(0, 3 * duracao[i])
        y_ruido_completo = data_matrix_ruido[inicio_ruido[i] - duracao[i]: inicio_ruido[i] + 2 * duracao[i]]
        

        #print ('%s minutos antes do evento: %s'%(duracao[i],data_hora[npontos[i] - duracao[i]]))
        #print (u'Início do evento: %s'%(data_hora[npontos[i]]))
        #print ('Fim do evento: %s'%(data_hora[npontos[i] + duracao[i]]))
        #print ('%s minutos após o fim do evento: %s'%(duracao[i],data_hora[npontos[i] + 2 * duracao[i]]))
    
        #plt.plot(x_ruido, y_ruido, label = 'Ruido (cor mais forte)', color = 'black')
        #plt.plot(x_hann, y_hann, linestyle = '--', label = u'Hann (durante o ruido)', color = 'blue')   
        plt.plot(x_ruido_completo, y_ruido_completo, color = 'red')
   
        plt.legend(loc = 'upper right', fontsize = 'xx-large')
     
        plt.title('Gerador de ruidos', fontsize = 20)
        plt.xlabel('Minutos', fontsize = 20)
        plt.ylabel(u'Vazão',  fontsize = 20)

        fig = plt.gcf()
        fig.set_size_inches(18.5, 12.5)

        plt.show() 

In [34]:
#plot_grafico(data_matrix_suavizado, data_matrix_ruido, inicio_ruido, duracao)

In [35]:
# Treino

X_treino = [pontos_negativos[0,], pontos_positivos[0,]]
y = [-1] * pontos_negativos.shape[1] + [1] * pontos_positivos.shape[1]
X_teste = [pontos_negativos[1,], pontos_positivos[1,]]

X_treino = np.array(X_treino).ravel()
y = np.array(y).ravel()
X_teste = np.array(X_teste).ravel()

X_treino = X_treino.reshape(-1,1)
X_teste = X_teste.reshape(-1,1)

params = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10, 100]}

random_search = RandomizedSearchCV(SVC(kernel = 'rbf'), param_distributions = params, cv = StratifiedKFold())

random_search.fit(X_treino, y)

[[ 27.        ]
 [ 27.44909879]
 [ 27.84402963]
 ..., 
 [ 24.        ]
 [ 18.        ]
 [ 18.        ]]
[-1 -1 -1 ...,  1  1  1]
12000
12000


In [37]:
# Teste

y_predict = random_search.predict(X_teste)

score_treino = random_search.best_score_

print y_predict

print score_treino

[-1 -1 -1 ...,  1  1  1]
0.924666666667


In [ ]:
plt.figure(num=None, figsize=(18.5, 12.5), dpi=80, facecolor='w', edgecolor='k')
plt.plot(X_teste, color = 'b', label = 'Target test')
plt.plot(y_predict, color='r', label='Predicted')

plt.title('SVM', fontsize = 20)
plt.xlabel('Minutos', fontsize = 20)
plt.ylabel(u'Vazão',  fontsize = 20)

plt.legend(loc = 'upper right', fontsize = 'xx-large')

#plt.show()